# Lab 5.1: Dashboard de Ventas Básico con SQL

**Resumen Ejecutivo**: Este laboratorio práctico te guía en la construcción de las consultas SQL fundamentales para alimentar un dashboard de ventas ejecutivo. Aprenderás a transformar datos transaccionales en métricas agregadas (KPIs) que responden preguntas de negocio críticas: *¿Cuánto vendimos? ¿Qué productos generan mayor margen? ¿Cuáles son las tendencias?* Este es el trabajo diario de un analista de BI: convertir datos en información accionable.

---

## 🎯 Objetivos de Aprendizaje

Al completar este laboratorio podrás:

1. **Agregar** datos transaccionales con `GROUP BY` para obtener métricas consolidadas
2. **Calcular** KPIs de negocio: ingresos, margen bruto, unidades vendidas
3. **Combinar** tablas dimensionales (`dim_productos`, `dim_clientes`) con tablas de hechos (`fact_ventas`)
4. **Optimizar** consultas para dashboards (evitar duplicidades, filtros eficientes)
5. **Estructurar** resultados listos para conectar con herramientas de visualización (Power BI, Tableau)

---

## 📋 Prerequisites

Antes de iniciar este lab, verifica:

- ✅ **Dataset cargado**: Archivo `dataset_setup.sql` ejecutado (crea tablas `fact_ventas`, `dim_productos`, `dim_clientes`, `dim_tiempo`)
- ✅ **Conocimientos previos**: 
  - Joins entre tablas (INNER JOIN, LEFT JOIN)
  - Funciones de agregación (`SUM`, `AVG`, `COUNT`)
  - Cláusula `GROUP BY` y `HAVING`
- ✅ **Herramientas**: SQL Server Management Studio (SSMS) o Azure Data Studio

**Verificación del dataset**:
```sql
-- Ejecutar para confirmar que tienes datos
SELECT COUNT(*) AS registros_ventas FROM dbo.fact_ventas;
-- Resultado esperado: >1000 registros
```

---

## 🛠️ Contexto de Negocio

### 🏢 Empresa: TechRetail S.A.

**Situación**: Eres analista de BI en una cadena de retail tecnológico. El CFO te solicita un dashboard que responda estas preguntas **cada mañana a las 8:00 AM**:

1. **¿Cuántos ingresos generamos ayer?** (KPI crítico para flujo de caja)
2. **¿Qué productos tienen el mayor margen bruto?** (optimización de mix de productos)
3. **¿Cuáles clientes son los más valiosos?** (estrategias de retención)
4. **¿Cuáles regiones tienen bajo rendimiento?** (reasignación de recursos de marketing)

**Tu tarea**: Crear las consultas SQL que alimentarán 4 visualizaciones del dashboard.

---

## 📊 Consulta 1: Métricas Diarias (Unidades, Ingresos, Margen)

### 🎯 Objetivo de Negocio

El CFO necesita ver la evolución diaria de 3 métricas clave:
- **Unidades vendidas**: Indicador de volumen operativo
- **Ingresos totales**: Facturación bruta (precio × cantidad × descuento)
- **Margen bruto**: Utilidad antes de gastos operativos (precio - costo)

### 📝 Explicación Técnica

Esta consulta:
1. **Une** tabla de hechos (`fact_ventas`) con dimensión de productos (`dim_productos`) para obtener costos
2. **Agrupa** por fecha para consolidar múltiples transacciones del mismo día
3. **Calcula** ingresos considerando descuentos: `precio × cantidad × (1 - descuento%/100)`
4. **Calcula** margen: `cantidad × (precio - costo)`

**Complejidad de la fórmula de ingresos**:
```
Ingreso Real = Cantidad × Precio Unitario × (1 - Descuento%)
Ejemplo: 10 unidades × $100 × (1 - 15%) = $850
```

In [ ]:
-- ============================================
-- CONSULTA 1: Métricas Diarias Consolidadas
-- ============================================
-- Resultado esperado: 1 fila por día con 3 KPIs

SELECT 
    fecha,
    SUM(cantidad) AS unidades_vendidas,
    SUM(cantidad * p.precio_unitario * (1 - descuento_pct/100.0)) AS ingresos_totales,
    SUM(cantidad * (p.precio_unitario - p.costo_unitario)) AS margen_bruto
FROM dbo.fact_ventas fv
JOIN dbo.dim_productos p ON fv.producto_id = p.producto_id
GROUP BY fecha
ORDER BY fecha DESC;

-- Interpretación:
-- - Si margen_bruto/ingresos_totales < 30%, revisar estrategia de descuentos
-- - Caída brusca en unidades_vendidas puede indicar problema de inventario

**📈 Uso en Dashboard**: Esta consulta alimenta un gráfico de líneas con 3 series temporales (unidades, ingresos, margen) para identificar tendencias y anomalías.

---

## 📊 Consulta 2: Top 10 Productos por Margen

### 🎯 Objetivo de Negocio

Identificar qué productos generan mayor **utilidad absoluta** (no solo facturación). Un producto puede vender mucho pero tener margen bajo (ej: commodities), mientras que un producto de nicho puede vender poco pero con margen alto.

**Decisión de negocio**: Enfocar marketing en productos de alto margen + volumen razonable.

In [ ]:
-- ============================================
-- CONSULTA 2: Top 10 Productos por Margen
-- ============================================
SELECT TOP 10
    p.nombre_producto,
    p.categoria,
    SUM(cantidad) AS unidades_vendidas,
    SUM(cantidad * (p.precio_unitario - p.costo_unitario)) AS margen_total,
    -- Calcular margen porcentual promedio
    AVG((p.precio_unitario - p.costo_unitario) / NULLIF(p.precio_unitario, 0) * 100) AS margen_pct
FROM dbo.fact_ventas fv
JOIN dbo.dim_productos p ON fv.producto_id = p.producto_id
GROUP BY p.nombre_producto, p.categoria
ORDER BY margen_total DESC;

-- Insight: Si un producto con margen_pct > 50% tiene pocas unidades, 
-- considerar campaña promocional para aumentar volumen

---

## 📊 Consulta 3: Clientes con Mayor Valor (Lifetime Value)

### 🎯 Objetivo de Negocio

Identificar los clientes "VIP" que generan más ingresos históricos. Estos clientes:
- Deben recibir **atención prioritaria** (soporte 24/7)
- Son candidatos para **programas de fidelización** premium
- Requieren **seguimiento proactivo** (prevenir churn)

**Regla 80/20**: Típicamente 20% de clientes generan 80% de ingresos.

In [ ]:
-- ============================================
-- CONSULTA 3: Top 20 Clientes por Valor Total
-- ============================================
SELECT TOP 20
    c.nombre_cliente,
    c.segmento,
    c.region,
    COUNT(DISTINCT fv.venta_id) AS num_transacciones,
    SUM(cantidad * p.precio_unitario * (1 - descuento_pct/100.0)) AS valor_total_cliente,
    AVG(cantidad * p.precio_unitario * (1 - descuento_pct/100.0)) AS ticket_promedio
FROM dbo.fact_ventas fv
JOIN dbo.dim_productos p ON fv.producto_id = p.producto_id
JOIN dbo.dim_clientes c ON fv.cliente_id = c.cliente_id
GROUP BY c.nombre_cliente, c.segmento, c.region
ORDER BY valor_total_cliente DESC;

-- Acción: Si num_transacciones > 50 y ticket_promedio > $500, 
-- asignar ejecutivo de cuenta dedicado

---

## 📊 Consulta 4: Rendimiento por Región Geográfica

### 🎯 Objetivo de Negocio

Comparar el desempeño de ventas entre regiones para:
- **Redistribuir presupuesto de marketing** hacia regiones con bajo rendimiento
- **Replicar estrategias exitosas** de regiones top performers
- **Identificar mercados saturados** vs mercados con potencial de crecimiento

In [ ]:
-- ============================================
-- CONSULTA 4: Ventas por Región
-- ============================================
SELECT 
    c.region,
    COUNT(DISTINCT c.cliente_id) AS num_clientes_activos,
    SUM(cantidad) AS unidades_totales,
    SUM(cantidad * p.precio_unitario * (1 - descuento_pct/100.0)) AS ingresos_region,
    -- Calcular ingreso promedio por cliente (ARPU)
    SUM(cantidad * p.precio_unitario * (1 - descuento_pct/100.0)) / 
        NULLIF(COUNT(DISTINCT c.cliente_id), 0) AS ingreso_promedio_por_cliente
FROM dbo.fact_ventas fv
JOIN dbo.dim_productos p ON fv.producto_id = p.producto_id
JOIN dbo.dim_clientes c ON fv.cliente_id = c.cliente_id
GROUP BY c.region
ORDER BY ingresos_region DESC;

-- KPI clave: ingreso_promedio_por_cliente (ARPU)
-- Si región A tiene ARPU bajo pero muchos clientes, optimizar cross-selling

---

## 💼 Valor de Negocio: Impacto de un Dashboard Ejecutivo

### 📊 Métricas de Adopción y Uso

**Antes del dashboard** (decisiones basadas en reportes manuales):
- ⏱️ **Tiempo para generar reporte semanal**: 4 horas (analista en Excel)
- 📧 **Frecuencia de actualización**: Semanal (datos obsoletos)
- 🎯 **Velocidad de respuesta a cambios**: 5-7 días

**Después del dashboard** (SQL + Power BI automatizado):
- ⏱️ **Tiempo para generar reporte**: 0 minutos (actualización automática cada hora)
- 📧 **Frecuencia de actualización**: Tiempo real (con actualización incremental)
- 🎯 **Velocidad de respuesta**: <24 horas (alertas automáticas)

### 💰 ROI Estimado

**Caso real: Retail mediano (50M USD facturación anual)**

| Beneficio | Ahorro Anual | Cálculo |
|-----------|--------------|---------|
| **Automatización de reportes** | $18,000 USD | 4 horas/semana × $15/hora × 52 semanas |
| **Detección temprana de problemas** | $125,000 USD | Evitar 1 mes de ventas con producto sin stock ($500K × 25% margen) |
| **Optimización de mix de productos** | $75,000 USD | Rebalanceo hacia productos de alto margen (+5% en margen global) |
| **Reducción de descuentos innecesarios** | $50,000 USD | Eliminar descuentos >30% sin justificación comercial |
| **TOTAL AHORRO ANUAL** | **$268,000 USD** | |

**Inversión requerida**:
- Desarrollo de dashboard: $15,000 USD (3 semanas analista senior)
- Licencias Power BI Pro: $2,400 USD/año (20 usuarios × $10/mes)
- **ROI Año 1**: **1,677%** ($268K ahorro / $15K inversión)

### 🎯 Decisiones de Negocio Habilitadas

Con este dashboard, el equipo directivo puede:

1. **Reaccionar a tendencias en tiempo real**: Ajustar inventario si producto top está agotándose
2. **Personalizar estrategias por región**: Aumentar presupuesto de marketing en regiones de bajo rendimiento
3. **Priorizar atención a clientes VIP**: Asignar ejecutivos de cuenta a top 20 clientes
4. **Optimizar descuentos**: Reducir descuentos en productos de alta demanda
5. **Proyectar flujo de caja**: Ingresos diarios → forecast semanal → planificación financiera

---

## 📌 Conclusiones y Próximos Pasos

### 🎓 Conceptos Aprendidos

1. **Agregación con GROUP BY** para consolidar datos transaccionales
2. **Cálculo de KPIs de negocio** (ingresos con descuento, margen bruto)
3. **Joins multitabla** para enriquecer información (productos, clientes, regiones)
4. **Optimización de consultas** para dashboards interactivos (<5 segundos)

### 🔑 Habilidades Desarrolladas

- Transformar requerimientos de negocio en consultas SQL ejecutables
- Validar resultados con casos de prueba y reconciliación contable
- Estructurar datos para conectar con herramientas de visualización
- Identificar y corregir errores comunes (duplicidades, divisiones por cero)

### 🚀 Evolución del Lab

Con estas consultas base puedes:

1. **Conectar a Power BI/Tableau**: Crear visualizaciones interactivas con drill-down
2. **Automatizar reportes**: Scheduled jobs con SQL Server Agent para envío por correo
3. **Añadir análisis predictivo**: Machine Learning con Python/R para forecasting
4. **Implementar alertas**: Notificaciones cuando KPIs caen bajo umbrales críticos

---

## 💡 Mejores Prácticas para Dashboards SQL

### ✅ Optimización de Performance

| Práctica | Beneficio | Implementación |
|----------|-----------|----------------|
| **Usar vistas materializadas** | Precálculo de agregaciones pesadas | `CREATE VIEW vw_ventas_diarias WITH SCHEMABINDING AS...` |
| **Índices en claves de join** | Reduce tiempo de consulta de minutos a segundos | `CREATE INDEX idx_producto_id ON fact_ventas(producto_id)` |
| **Particionar tablas grandes** | Consultas solo leen particiones relevantes | Particionar `fact_ventas` por año |
| **Filtrar por fechas indexadas** | Evita full table scan | `WHERE fecha >= DATEADD(day, -30, GETDATE())` |
| **Limitar resultados con TOP** | Reduce transferencia de datos | `SELECT TOP 100...` para tablas de millones de registros |

### ✅ Calidad de Datos

- **Validar datos antes de agregar**: Verificar que no haya `NULL` en campos críticos (`ISNULL(cantidad, 0)`)
- **Consistencia en cálculos**: Centralizar fórmulas en funciones o CTEs (evitar duplicar lógica)
- **Auditoría de totales**: Comparar suma de detalle vs totales contables
- **Manejo de divisiones por cero**: Siempre usar `NULLIF(denominador, 0)`

### ✅ Presentación de Resultados

- **Ordenar por relevancia**: Métricas críticas primero (`ORDER BY ingresos_totales DESC`)
- **Formatear números**: Usar `FORMAT(ingresos, 'C', 'es-CO')` para moneda local
- **Incluir metadatos**: Fecha de última actualización, fuente de datos

---

## 🚀 Ejercicios Prácticos

### 🟢 Ejercicio 1: Análisis Mensual de Ventas

**Objetivo**: Modificar la Consulta 1 para mostrar métricas agregadas por **mes** en lugar de por día.

**Pistas**:
- Usar `FORMAT(fecha, 'yyyy-MM')` o `YEAR(fecha)` + `MONTH(fecha)`
- Agregar columna con nombre de mes (`DATENAME(month, fecha)`)

**Resultado esperado**: 1 fila por mes con unidades, ingresos y margen.

---

### 🟠 Ejercicio 2: Productos con Margen Negativo

**Objetivo**: Identificar productos que se están vendiendo **con pérdida** (margen < 0).

**Posibles causas**:
- Descuentos excesivos (>100%)
- Costo unitario mal configurado
- Promociones sin análisis de rentabilidad

**Acción**: Generar reporte para equipo de pricing.

---

### 🔴 Ejercicio 3: Segmentación RFM de Clientes

**Objetivo**: Clasificar clientes según modelo **Recency, Frequency, Monetary** (RFM).

**Métricas a calcular**:
- **Recency**: Días desde última compra
- **Frequency**: Número de transacciones
- **Monetary**: Valor total de compras

**Resultado**: Segmentos (VIP, Regulares, En Riesgo, Inactivos) para estrategias diferenciadas de marketing.

---

## ⚠️ Errores Comunes y Cómo Evitarlos

| Error | Síntoma | Causa | Solución |
|-------|---------|-------|----------|
| **Duplicación de registros** | Ingresos inflados (ej: $10M en vez de $5M) | Joins incorrectos (productos duplicados en dim_productos) | Verificar unicidad de claves con `COUNT(DISTINCT producto_id)` |
| **División por cero** | Error `Divide by zero` | `precio_unitario = 0` en datos | Usar `NULLIF(precio_unitario, 0)` en denominadores |
| **Filtros inconsistentes** | Comparaciones no alineadas | Usar `fecha >= '2024-01-01'` en una consulta y `fecha > '2024-01-01'` en otra | Centralizar definición de periodos con CTE |
| **Descuentos mal calculados** | Margen negativo | Descuento aplicado 2 veces (en precio y en margen) | Aplicar descuento **solo** en cálculo de ingresos |
| **Performance lenta** | Consulta >10 segundos | Falta de índices en claves de join | Crear índices en `producto_id`, `cliente_id`, `fecha` |

### 🔍 Checklist de Validación

Antes de entregar la consulta al dashboard:

- [ ] **Ejecutar con datos de prueba conocidos** (verificar que 2+2=4)
- [ ] **Comparar totales con reporte financiero oficial** (reconciliación contable)
- [ ] **Probar con diferentes rangos de fechas** (incluir casos extremos: 1 día, 1 año)
- [ ] **Verificar que `NULL` no distorsiona agregaciones** (usar `ISNULL` o `COALESCE`)
- [ ] **Medir tiempo de ejecución** (objetivo: <5 segundos para dashboards interactivos)

---

## Navegación

[⬅️ Volver al Índice del Curso](../README.md) | [Siguiente Lab: Optimización de Consultas ➡️](02_optimizacion_basico.ipynb)